# 02 GEE Map Greenland
This is a simple exerciese of mapping Greenland with Google Earth Engine.

In [1]:
# import pandas as pd
import ee
# if in colab:
# !pip install geemap
import geemap
import geemap.colormaps as cm
# import geemap.eefolium as geemap
# ee.Initialize()

## Greenland: Arctic DEM

In [2]:
Map = geemap.Map()
Map # comment this line if in colab

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:


greenlandmask = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK') \
                   .select('ice_mask').eq(1); #'ice_mask', 'ocean_mask'
arcticDEM = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic')

arcticDEMgreenland = arcticDEM.updateMask(greenlandmask)

palette = cm.get_palette('terrain', n_class=10)

visPara = {'min': 0,  'max': 2500.0, 'palette': ['0d13d8', '60e1ff', 'ffffff']}
# visPara = {'min': 0,  'max': 2500.0, 'palette': palette}
Map.addLayer(arcticDEMgreenland, visPara, 'Arctic DEM')
Map.setCenter(-41.0, 74.0, 3)
#add colorbar
colors = visPara['palette']
vmin = visPara['min']
vmax = visPara['max']

Map.add_colorbar(visPara, label="Elevation (m a.s.l)", orientation="vertical", layer_name="Arctic DEM")
# Map.add_colorbar_branca(colors=colors, vmin=vmin, vmax=vmax, caption="m a.s.l")


## Greenland: ArcticDEM terrain palette

In [4]:
Map1 = geemap.Map()
Map1 # comment this line if in colab

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
greenlandmask = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK') \
                   .select('ice_mask').eq(1); #'ice_mask', 'ocean_mask'
arcticDEM = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic')

arcticDEMgreenland = arcticDEM.updateMask(greenlandmask)

palette = cm.get_palette('terrain', n_class=10)

# visPara = {'min': 0,  'max': 2500.0, 'palette': ['0d13d8', '60e1ff', 'ffffff']}
visPara = {'min': 0,  'max': 2500.0, 'palette': palette}

Map1.addLayer(arcticDEMgreenland, visPara, 'Arctic DEM terrain')
Map1.setCenter(-41.0, 74.0, 3)
Map1.add_colorbar(visPara, label="Elevation (m)", discrete=False, orientation="vertical", layer_name="Arctic DEM terrain")

## Greenland: ArcticDEM contour
ref: https://twitter.com/jstnbraaten/status/1372958857266229252

In [6]:
Map2 = geemap.Map()
Map2 # comment this line if in colab

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [7]:
greenlandmask = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK') \
                   .select('ocean_mask').eq(0); #'ice_mask', 'ocean_mask'
arcticDEM = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic')

arcticDEMgreenland = arcticDEM.updateMask(greenlandmask)
Map.setCenter(-41.0, 74.0, 3)

mask = arcticDEMgreenland.gt(0)

elevZones = arcticDEMgreenland.expression(
  "(b('elevation') > 2500) ? 10" + \
  ": (b('elevation') > 2250) ? 9" + \
  ": (b('elevation') > 2000) ? 8" + \
  ": (b('elevation') > 1750) ? 7" + \
  ": (b('elevation') > 1500) ? 6" + \
  ": (b('elevation') > 1250) ? 5" + \
  ": (b('elevation') > 1000) ? 4" + \
  ": (b('elevation') > 750) ? 3" + \
  ": (b('elevation') > 500) ? 2" + \
  ": (b('elevation') > 250) ? 1" + \
  ": 0"
).mask(mask)

palette = cm.get_palette('terrain', n_class=10)
visPara = {'min': 0,  'max': 2500.0, 'palette': palette}


# Make a height layer from the binned elevation.
height = elevZones.multiply(175)

# Cast shadows based on elevation bin height.
shadow = (ee.Terrain.hillShadow(
  height.updateMask(mask).unmask(0), 310, 70, 30, False).Not().selfMask()) \
  .visualize(**{'min': 1, 'max': 1, 'palette': '000', 'opacity': 0.9})

# Add a palette to the elevation bins.
elevZonesVis = elevZones.visualize(**{'min': 0, 'max': 10, 'palette': palette})
bg = ee.Image(0).visualize(**{'palette': 'FFF'})
paper = ee.Terrain.hillshade(
  ee.Image.random(0).reproject('EPSG:4326', None, 500).multiply(1000)) \
  .visualize(**{'palette': ['A0A0A0', 'fff'], 'opacity': 0.07})


composite = bg.blend(elevZonesVis).blend(shadow).blend(paper)

Map2.setCenter(-41.0, 74.0, 3)
Map2.addLayer(composite, name='ArcticDEM contour')
Map2.add_colorbar(visPara, label="Elevation (m)", discrete=True, orientation="vertical", layer_name="ArcticDEM contour")